In [1]:
import pandas as pd

In [19]:
import json
import os
import csv

In [3]:
os.getcwd()

'/repos/EHDP-data/indicators'

In [12]:
test = json.load(open("data/2120.json"))

In [13]:
df = pd.DataFrame.from_dict(test)

In [36]:
def read_csv_as_index(csv_file):
    index = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            index[row['GeoID']] = row
    return index

In [21]:
csv_file = '../geography/GeoLookup.csv'

In [25]:
topojson_file = '../geography/NTA_2020.topo.json'

In [157]:
file_list = ['data/2120.json', 'data/2121.json', 'data/2122.json', 'data/2124.json', 'data/2132.json']

In [175]:
json_file = 'data/2124.json'
output_file = 'joined_data_2124.topojson'

def join_json_to_topojson(json_file, topojson_file, csv_file, output_file):
    # Step 1: Read the CSV file and store index values
    index = read_csv_as_index(csv_file)

    # Step 2: Read the JSON file
    with open(json_file, 'r') as file:
        json_data = json.load(file)

    # Step 3: Read the TopoJSON file
    with open(topojson_file, 'r') as file:
        topojson_data = json.load(file)

    # Step 4: Iterate through objects in TopoJSON
    for object_key in topojson_data['objects']:
        object_data = topojson_data['objects'][object_key]
        if object_data['type'] == 'GeometryCollection':
            # Iterate through geometries in GeometryCollection
            for geometry in object_data['geometries']:
                # Step 5: Retrieve corresponding data from JSON using index values
                id_value = geometry['properties']['id']
                if id_value in index:
                    json_entry = index[id_value]
                    json_entry['value'] = json_data[int(id_value)]['Value']
                    json_entry['measure_id'] = json_data[int(id_value)]['MeasureID']
                    json_entry['time'] = json_data[int(id_value)]['Time']
                    # Step 6: Add retrieved data to TopoJSON properties
                    geometry['properties'] = json_entry

    # Step 7: Write updated TopoJSON data to a new file
    with open(output_file, 'w') as file:
        json.dump(topojson_data, file)

In [176]:
join_json_to_topojson(json_file, topojson_file, csv_file, output_file)
